In [104]:
import json
import re
import pandas as pd
import warnings
from natasha import Doc, Segmenter, MorphVocab, NewsEmbedding,\
    NewsMorphTagger, NewsNERTagger

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.cluster import KMeans

import pymorphy2
analyzer = pymorphy2.MorphAnalyzer()
warnings.simplefilter('ignore')

segmenter = Segmenter()
embedding = NewsEmbedding()
morph_tagger = NewsMorphTagger(embedding)
morph_vocab = MorphVocab()
ner_tagger = NewsNERTagger(embedding)

### Преобразуем корпус в датафрейм

In [105]:
with open('dict_corpus.json', 'r') as f:
    corpus = json.load(f)

titles = list(corpus.keys())
texts = list(corpus.values())
df = pd.DataFrame.from_dict({'title':titles, 'text':texts})

In [106]:
df.head()

,title,text
0,0,Миф об обязательных вибрациях при выходе из те...
1,1,Где находится астрал и почему я его не вижу ...
2,2,"Прямой метод - это техника входа в фазу, осозн..."
3,3,"Виды техник осознанного сновидения Даже тот,..."
4,4,Почему люди ходят во сне Вопрос «Куда нас ув...


### Функция для сегментации, токенизации и лемматизации текста с последующим выделением именованных сущностей.

In [140]:
stop_w_entities = {'Здравствуйте'}

def get_ner(transcript):
    script = Doc(re.sub(r'\((.*?)\)', "", transcript))
    script.segment(segmenter)
    script.tag_morph(morph_tagger)
    for token in script.tokens:
        token.lemmatize(morph_vocab)
    script.tag_ner(ner_tagger)
    for span in script.spans:
        span.normalize(morph_vocab)
    named_ents = [(i.text, i.type, i.normal) for i in script.spans]
    normed_ents = []
    for word, tag, norm in named_ents:
        if word not in stop_w_entities and analyzer.parse(word.lower())[0].tag.POS == 'NOUN':
            if len(word.split()) == 1 and tag == "LOC":
                for gram in range(len(analyzer.parse(word))):
                    if "Geox" in analyzer.parse(word)[gram].tag:
                        normed_ents.append((analyzer.parse(word)[gram].normal_form))
                        break
                    elif gram == len(analyzer.parse(word)) - 1:
                        normed_ents.append((norm.lower().strip(".,!?;-")))
            else:
                normed_ents.append((norm.lower().strip(".,!?;-")))
    return sorted(normed_ents)

In [141]:
df["named_entities"] = df.apply(lambda row: get_ner(row["text"]), axis=1)
has_ner = [i for i in df.index.values if df.named_entities[i]]
df_ner = df[df.index.isin(has_ner)]
print(len(has_ner), df.shape[0])
df_ner

106 278


,title,text,named_entities
0,0,Миф об обязательных вибрациях при выходе из те...,"[вибрация, прямой метод]"
1,1,Где находится астрал и почему я его не вижу ...,"[астрал, прямой метод]"
2,2,"Прямой метод - это техника входа в фазу, осозн...","[вто, прямой метод, прямой метод, роберт брюс,..."
3,3,"Виды техник осознанного сновидения Даже тот,...","[прямой метод, степан юсин]"
4,4,Почему люди ходят во сне Вопрос «Куда нас ув...,"[бдг-фазе, билтон, верховный суд, вопрос, джей..."
...,...,...,...
261,261,Где находится астрал и почему я его не вижу ...,"[акаши, астрал, хроники акаши]"
264,264,Об исследовании выхода из тела Существует бол...,"[голландский институт мозг, дик свааба]"
267,267,Здравствуйте Задумывались ли вы когда-нибудь ...,[таланты]
275,275,"Вчера был день рождения у лидер группы ""Пилот""...","[кнабенгоф, пилот]"


### Итак, имеем датафрейм, где для каждого документа выделены именованные сущности

In [143]:
ner_voc = []
for row in df_ner.named_entities.tolist():
    ner_voc.extend(row)
print(f'Всего сущностей: {len(ner_voc)}\nУникальных: {len(set(ner_voc))}')

Всего сущностей: 460
Уникальных: 322


In [144]:
vocabulary = sorted(set(ner_voc))
corpus = df_ner.named_entities.apply(str).tolist()

In [161]:
pipe = Pipeline([('count', CountVectorizer(vocabulary=vocabulary)),
                 ('tfid', TfidfTransformer())]).fit(corpus)
X = pipe.fit_transform(corpus)
km = KMeans(n_clusters=10, init='k-means++', max_iter=600,
            algorithm="full", precompute_distances=True)
km.fit(X)

KMeans(algorithm='full', max_iter=600, n_clusters=10, precompute_distances=True)

### Посмотрим на внутренние метрики
- **Силуэт: показывает, насколько объект похож на свой кластер относительно других кластеров; если значение стремится к 1 - хорошее разбиение, если к -1 - плохое, если в районе 0 - кластеры пересекаются. В нашем случае кластеры пересекаются, но разбиение скорее хорошее**
- **Индекс Дэвиcа-Болдуина: оценивает расстояние от объекта кластера до центроида и расстояние между центроидами; чем ниже, тем лучше разбиение. В нашем случае разбиение хорошее (может быть намного больше)**
#### С увеличением количества кластеров увеличивается силуэт и уменьшается индекс. Остановимся на 30 кластерах

In [162]:
print(metrics.silhouette_score(X, km.labels_, sample_size=1000))
print(metrics.davies_bouldin_score(X.toarray(), km.labels_))

0.21943493781997742
1.2228413841025365


In [163]:
km = KMeans(n_clusters=20, init='k-means++', max_iter=600,
            algorithm="full", precompute_distances=True)
km.fit(X)
print(metrics.silhouette_score(X, km.labels_, sample_size=1000))
print(metrics.davies_bouldin_score(X.toarray(), km.labels_))

0.24983934234116253
0.935093192487067


In [164]:
km = KMeans(n_clusters=25, init='k-means++', max_iter=600,
            algorithm="full", precompute_distances=True)
km.fit(X)
print(metrics.silhouette_score(X, km.labels_, sample_size=1000))
print(metrics.davies_bouldin_score(X.toarray(), km.labels_))

0.26120678048994656
0.8867571013175042


In [165]:
km = KMeans(n_clusters=30, init='k-means++', max_iter=600,
            algorithm="full", precompute_distances=True)
km.fit(X)
print(metrics.silhouette_score(X, km.labels_, sample_size=1000))
print(metrics.davies_bouldin_score(X.toarray(), km.labels_))

0.26192927717614617
0.8313719982054736


In [166]:
df_ner["label"] = km.predict(X)

In [167]:
df_ner["label"].value_counts()

3     58
4      5
20     4
7      4
5      3
11     2
16     2
14     2
13     2
12     2
10     2
9      2
21     1
26     1
27     1
25     1
28     1
24     1
23     1
22     1
0      1
15     1
19     1
18     1
17     1
1      1
8      1
6      1
2      1
29     1
Name: label, dtype: int64

### В кластере 3 оказалось большинство объектов

In [169]:
df_ner.query("label == 3").sample(5)

,title,text,named_entities,label
71,71,Как осознанность влияет на реальную жизнь Дол...,"[джон кабат-зинн, дэвидсон, дэвидсон, дэвидсон...",3
156,156,Чем выход из тела отличается от осознанного сн...,"[китай, лента]",3
248,248,Приглашаю вас на открытый вебинар! Уже сегодня...,[мария егорова],3
114,114,Какой сон вещий и как можно научиться видеть т...,"[авраам линкольн, белый дом, белый дом, белый ...",3
102,102,"Как засыпать за 2 минуты, где бы вы ни находил...","[бад уинтер, уинтер, уинтер, уинтер]",3


In [170]:
df_ner.query("label == 4")

,title,text,named_entities,label
26,26,Сегодня я хочу вам порекомендовать книгу по ос...,"[лаберж, элис робб]",4
34,34,Как увидеть осознанный сон. Техника вдвойне эф...,[лаберж],4
79,79,"Как увидеть осознанный сон, если хорошо помниш...",[лаберж],4
217,217,ТОП-3 книги для новичков По осознанным снам...,"[вадим зеланд, патриция гарфилд, сеноев, стиве...",4
221,221,Миф об обязательных вибрациях при выходе из те...,[стивен лаберж],4


In [171]:
df_ner.query("label == 7")

,title,text,named_entities,label
3,3,"Виды техник осознанного сновидения Даже тот,...","[прямой метод, степан юсин]",7
86,86,Степан Юсин. Сонный паралич. 21.12.2020.,[степан юсин],7
125,125,Осознанные сны подписчиков Вчера у меня началс...,"[осы, степан]",7
142,142,"Свежий отзыв на мою книгу от ""Прочитал книг...","[степан юсин, фазер]",7


In [172]:
df_ner.query("label == 20")

,title,text,named_entities,label
85,85,Друг или враг: почему нам снятся кошмары? Не...,[сон],20
190,190,Ложное пробуждение ОС наших подписчиков Хот...,"[исс, сон]",20
191,191,Как найти кошелёк во сне. Осознанные сны подп...,[сон],20
198,198,Таблеточки Осознанные сны наших подписчиков ...,"[билли айлиш, рубик, сон, федункив марина]",20


In [174]:
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
terms = pipe[0].get_feature_names()
for i in range(30):
    print("Cluster %d:" % i, end='')
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind], end='')
    print()

Cluster 0: апноэ терапевты я-наблюдатель калифорнийский университет иэн уоллес йорк кабат-зинн калеи калея калигари
Cluster 1: собакен люпин я-наблюдатель калифорнийский университет исс иэн уоллес йорк кабат-зинн калеи калея
Cluster 2: бузунов москва я-наблюдатель каран радж иэн уоллес йорк кабат-зинн калеи калея калигари
Cluster 3: австралия вьетнам менделеев подписчица подписка бзззз уинтер астрал привет группа
Cluster 4: лаберж сеноев я-наблюдатель калифорнийский университет исс иэн уоллес йорк кабат-зинн калеи калея
Cluster 5: фрейд мелехин нолан кинематографисты калигари москва йорк кабат-зинн калеи калея
Cluster 6: европа россия сша лаберж я-наблюдатель калигари иэн уоллес йорк кабат-зинн калеи
Cluster 7: степан фазер осы я-наблюдатель калифорнийский университет иэн уоллес йорк кабат-зинн калеи калея
Cluster 8: чай лосось я-наблюдатель калигари исс иэн уоллес йорк кабат-зинн калеи калея
Cluster 9: элайджа дело сон я-наблюдатель исс иэн уоллес йорк кабат-зинн калеи калея
Cluster 1

## Вывод
### Из последней визуализации кластеров нельзя точно сказать, что какие-то из них имеют особую тематику, т.к. многие сущности оказались в большинстве кластеров одовременно. Такое разбиение происходит при любом количестве кластеров. Отчетливо видны топонимы в кластерах 3, 6, 10, 26